## Test Marqo Model

In [ ]:
import sys
import os


# 获取当前文件的目录
current_dir = os.path.dirname(os.path.abspath("__file__"))

# 获取上一级目录
parent_dir = os.path.dirname(current_dir)

# 将上一级目录添加到 sys.path
sys.path.append(parent_dir)

from transformers import AutoModel, AutoProcessor
import torch
from PIL import Image
import requests
import matplotlib.pyplot as plt

from train.marqo_fashionSigLIP import MarqoFashionSigLIP, MarqoFashionSigLIPProcessor

model_name = "/data/model/Marqo/marqo-ecommerce-embeddings-L"
# model_name = 'Marqo/marqo-ecommerce-embeddings-B'

model = MarqoFashionSigLIP.from_pretrained(model_name)
processor = MarqoFashionSigLIPProcessor.from_pretrained(model_name)

print("Finish Loading Model")

model

# img = Image.open(
#     requests.get(
#         "https://raw.githubusercontent.com/marqo-ai/marqo-ecommerce-embeddings/refs/heads/main/images/dining-chairs.png",
#         stream=True,
#     ).raw
# ).convert("RGB")
# image = [img]
# text = ["dining chairs", "a laptop", "toothbrushes"]
# processed = processor(
#     text=text, images=image, padding="max_length", return_tensors="pt"
# )
# processor.image_processor.do_rescale = False
# with torch.no_grad():
#     image_features = model.get_image_features(processed["pixel_values"], normalize=True)
#     text_features = model.get_text_features(processed["input_ids"], normalize=True)

#     text_probs = (100 * image_features @ text_features.T).softmax(dim=-1)

# print(text_probs)
# [1.0000e+00, 8.3131e-12, 5.2173e-12]


In [13]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms


class ShopeeDataset(Dataset):
    def __init__(self, df, img_dir, split: str = "train"):
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                # convert to RGB
                transforms.Lambda(lambda img: img.convert("RGB")),
                transforms.ToTensor(),
            ]
        )
        self.split = split
        self.len = len(self.df)
        # self.imgs = self._read_all_images()

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        if self.split == "train":
            query_text = row["query"]
            pos_text = row["pos_txt"][0]
            neg_text = row["neg_txt"][0]

            query_img_path = os.path.join(self.img_dir, row["image"])
            pos_img_path = os.path.join(self.img_dir, row["pos_img"][0])
            neg_img_path = os.path.join(self.img_dir, row["neg_img"][0])

            query_img = self._get_image(query_img_path)
            pos_img = self._get_image(pos_img_path)
            neg_img = self._get_image(neg_img_path)

            return {
                "query": {
                    "text": query_text,
                    "image": query_img,
                },
                "pos": {
                    "text": pos_text,
                    "image": pos_img,
                },
                "neg": {
                    "text": neg_text,
                    "image": neg_img,
                },
            }

        elif self.split == "valid":
            title = row["title"]
            image_path = os.path.join(self.img_dir, row["image"])

            img = self._get_image(image_path)
            pil_img = Image.open(image_path)

            return {
                "title": title,
                "image": img,
                "pil_image": pil_img,
                "image_path": image_path,
            }

    def _get_image(self, path):
        img = Image.open(path)
        img = self.transform(img)
        return img

    def get_pil_image(self, path):
        return Image.open(path)


def get_collate_fn(processor):
    def collate_fn(batch):
        images = [item["image"] for item in batch]
        texts = [item["title"] for item in batch]

        processed = processor(
            text=texts,
            images=images,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        processor.image_processor.do_rescale = False

        return processed

    return collate_fn


def visualize(df, index, col):
    """可视化图片和预测结果，预测结果为posting_id的list"""
    row = df.iloc[index]
    preds = row[col]
    img_dir: str = "../data/train_images"
    images = [df[df.posting_id == pred].image.values[0] for pred in preds]

    target_title = row.title
    target_img = row.image

    titles = [df[df.posting_id == pred].title.values[0] for pred in preds]
    images = [Image.open(os.path.join(img_dir, img)) for img in images]

    rows = 5
    cols = 5
    fig, ax = plt.subplots(rows, cols, figsize=(20, 20))

    plt.axis("off")

    for i in range(cols):
        ax[0, i].axis("off")

    for i in range(rows):
        ax[i, 0].axis("off")

    ax[0, 0].imshow(Image.open(os.path.join(img_dir, target_img)))
    ax[0, 0].set_title(
        "\n".join([target_title[i : i + 10] for i in range(0, len(target_title), 10)]),
        fontsize=12,
    )

    for i in range(1, rows):
        for j in range(1, cols):
            idx = (i - 1) * cols + (j - 1)
            ax[i, j].axis("off")
            if idx < len(images):
                ax[i, j].imshow(images[idx])
                ax[i, j].set_title(
                    "\n".join(
                        [
                            titles[idx][k : k + 10]
                            for k in range(0, len(titles[idx]), 10)
                        ]
                    ),
                    fontsize=12,
                )


def compute_f1(col):
    def f1(row):
        n = len(np.intersect1d(row["label"], row[col]))
        return 2 * n / (len(row["label"]) + len(row[col]))

    return f1


def compute_recall(col):
    def recall(row):
        n = len(np.intersect1d(row["label"], row[col]))
        return n / len(row["label"])

    return recall


def compute_precision(col):
    def precision(row):
        n = len(np.intersect1d(row["label"], row[col]))
        return n / len(row[col])

    return precision


def compute_precision_K(col, K):
    def precision(row):
        n = len(np.intersect1d(row["label"], row[col][:K]))
        return n / K

    return precision


def compute_AP(col, N):
    """compute average precision"""

    def AP(row):
        n = len(np.intersect1d(row["label"], row[col]))
        max_n = min(len(row[col]), N)
        if n == 0:
            return 0
        return (
            sum(
                [
                    compute_precision_K(col, i)(row)
                    for i in range(1, max_n + 1)
                    if row[col][i - 1] in row["label"]
                ]
            )
            / max_n
        )

    return AP


def retrieval(embs, df, chunk_size=4096, threshold=None, topK=None):
    assert (
        threshold is not None or topK is not None
    ), "Either threshold or topK should be provided"
    assert (
        threshold is None or topK is None
    ), "Only one of threshold or topK should be provided"

    embs_pt = torch.tensor(embs).cuda()

    num_chunks = (embs_pt.shape[0] + chunk_size - 1) // chunk_size
    posting_id = df.posting_id.to_list()
    topk_posting_id = []

    print(f"Chunk Size: {chunk_size}, {num_chunks} chunks")

    for i in tqdm(range(num_chunks)):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, embs_pt.shape[0])
        sim = embs_pt[start:end] @ embs_pt.T

        if topK is not None:
            indices = torch.topk(sim, topK, dim=1).indices.cpu().numpy()
            topk_posting_id.extend([[posting_id[j] for j in row] for row in indices])
        elif threshold is not None:
            mask = sim > threshold
            indices = [
                torch.nonzero(mask[j]).squeeze().cpu().numpy()
                for j in range(mask.shape[0])
            ]
            indices = [np.unique(i) for i in indices]
            sorted_indices = [
                indices[j][np.argsort(-sim[j, indices[j]].cpu().numpy())]
                for j in range(len(indices))
            ]
            topk_posting_id.extend(
                [[posting_id[j] for j in row] for row in sorted_indices]
            )

    # clean up
    del embs_pt
    torch.cuda.empty_cache()

    return topk_posting_id

In [6]:
from torch.utils.data import DataLoader

df = pd.read_csv("/data/MMRetrieval/data/train.csv")
img_dir = "/data/MMRetrieval/data/train_images"
split = "valid"

dataset = ShopeeDataset(df, img_dir, split)
collate_fn = get_collate_fn(processor)

dataloader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=False,
    num_workers=16,
    collate_fn=collate_fn,
)

In [ ]:
from tqdm import tqdm

embs = []

model.eval()
model.cuda()
for batch in tqdm(dataloader):
    with torch.no_grad():
        for k, v in batch.items():
            batch[k] = v.cuda() if isinstance(v, torch.Tensor) else v

        with torch.amp.autocast("cuda", torch.float16):
            image_features = model.get_image_features(
                batch["pixel_values"], normalize=True
            )
            text_features = model.get_text_features(batch["input_ids"], normalize=True)

        emb = torch.cat((image_features, text_features), dim=1).detach().cpu().numpy()
        emb /= np.linalg.norm(emb, axis=1, keepdims=True)

        embs.append(emb)

embs = np.concatenate(embs, axis=0)

del model, processor
torch.cuda.empty_cache()

In [ ]:
threshold = 0.73

for threshold in np.linspace(0.7, 0.8, 20):
    df["marqo"] = retrieval(embs, df, threshold=threshold, chunk_size=4096)

    tmp = df.groupby("label_group").posting_id.agg("unique").to_dict()
    df["label"] = df.label_group.map(tmp)

    df["f1"] = df.apply(compute_f1("marqo"), axis=1)
    df["recall"] = df.apply(compute_recall("marqo"), axis=1)
    df["precision"] = df.apply(compute_precision("marqo"), axis=1)
    df["AP"] = df.apply(compute_AP("marqo", 50), axis=1)

    print(f"Threshold: {threshold}")
    print(f"F1: {df.f1.mean()}")
    print(f"Recall: {df.recall.mean()}")
    print(f"Precision: {df.precision.mean()}")
    print(f"AP@50: {df.AP.mean()}")

In [ ]:
index = 1
samples = df[df["f1"] < 0.6].index.to_list()
visualize(df, samples[index], "marqo")
visualize(df, samples[index], "label")